In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import numpy as np
import os
import sys
scriptpath = "./import_data"
# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(scriptpath))
# Do the import
import import_data

In [ ]:
# Import data
all_data = import_data.import_data()
first_set_array = all_data[0]
second_set_array = all_data[1]
third_set_array = all_data[2]
fourth_set_array = all_data[3]
fifth_set_array = all_data[4]
sixth_set_array = all_data[5]
seventh_set_array = all_data[6]
eight_set_array = all_data[7]

In [ ]:
all_text = [i for i in first_set_array[1:,2]] \
+ [i for i in second_set_array[1:,2]] + [i for i in third_set_array[1:,2]] \
+ [i for i in fourth_set_array[1:,2]] + [i for i in fifth_set_array[1:,2]] \
+ [i for i in sixth_set_array[1:,2]] + [i for i in seventh_set_array[1:,2]] \
+ [i for i in eight_set_array[1:,2]]

all_set_index = [i for i in first_set_array[1:,1]] + [i for i in second_set_array[1:,1]] \
+ [i for i in second_set_array[1:,1]] + [i for i in third_set_array[1:,1]] \
+ [i for i in fourth_set_array[1:,1]] + [i for i in fifth_set_array[1:,1]] \
+ [i for i in sixth_set_array[1:,1]] + [i for i in seventh_set_array[1:,1]] \
+ [i for i in eight_set_array[1:,1]]

all_first_domain = [i for i in first_set_array[1:,6]] + [i for i in second_set_array[1:,6]] \
    + [i for i in third_set_array[1:,6]] + [i for i in fourth_set_array[1:,6]] \
    + [i for i in fifth_set_array[1:,6]] + [i for i in sixth_set_array[1:,6]] \
    + [i for i in seventh_set_array[1:,6]] + [i for i in eight_set_array[1:,6]]


scaler = preprocessing.StandardScaler()
all_first_domain = pd.DataFrame(all_first_domain)
all_first_domain_stnd = scaler.fit_transform(all_first_domain)
all_first_domain_stnd

# shuffle the data
# Given list1 and list2
all_text_shuf = []
all_first_domain_shuf = []
all_set_index_shuf = []
index_shuf = list(range(len(all_first_domain)))
shuffle(index_shuf)
for i in index_shuf:
  all_text_shuf.append(all_text[i])
  all_set_index_shuf.append(all_set_index[i])
  all_first_domain_shuf.append(all_first_domain_stnd[i])

# all_text_train = all_text[0:9000]
all_text_train = all_text_shuf[0:seventy_percent-1]
# all_text_train_labels = all_first_domain_stnd[0:9000]
all_text_train_labels = all_first_domain_shuf[0:seventy_percent-1]
all_text_train_sets = all_set_index_shuf[0:seventy_percent-1]
# all_text_test_labels = all_first_domain_stnd[9001:]
all_text_test_labels = all_first_domain_shuf[seventy_percent:]
# all_text_test = all_text[9001:]
all_text_test = all_text_shuf[seventy_percent:]
all_text_sets_sets = all_set_index_shuf[seventy_percent:]

all_text_train = [str(elem) for elem in all_text_train]
data_train1 = [all_text_train, all_text_train_labels, all_text_train_sets]
data_train1 =[list(i) for i in zip(*data_train1)]
len(all_text_train)
len(all_text_test)

# Now we are going to fit BERT into this new sets with 4 labels

all_text_test = [str(elem) for elem in all_text_test]
data_test1 = [all_text_test, all_text_test_labels, all_text_sets_sets]
data_test1 =[list(i) for i in zip(*data_test1)]

cols = ['essay', 'clarity', 'set']
train = pd.DataFrame.from_records(data_train1, columns=cols)
test = pd.DataFrame.from_records(data_test1, columns=cols)

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, input_ids, input_mask, segment_ids, label_id):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, shut_up=False):
  """Loads a data file into a list of `InputBatch`s."""

#   label_map = {}
#   for (i, label) in enumerate(label_list):
#     label_map[label] = i

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        segment_ids.append(1)
      tokens.append("[SEP]")
      segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
      input_ids.append(0)
      input_mask.append(0)
      segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    # label_id = label_map[example.label]
    if not shut_up:
        label_id = example.label
        if ex_index < 5:
          tf.logging.info("*** Example ***")
          tf.logging.info("guid: %s" % (example.guid))
          tf.logging.info("tokens: %s" % " ".join(
              [tokenization.printable_text(x) for x in tokens]))
          tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
          tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
          tf.logging.info(
              "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
          tf.logging.info("label: {} (id = {})".format(example.label, label_id))

    features.append(
        InputFeatures(
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids,
            label_id=label_id))
  return features

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN][0], set_ = x[SET_COLUMN]), axis = 1)
                                                                   

test_InputExamples = test.apply(lambda x: InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN][0], set_=x[SET_COLUMN]), axis = 1)

# We'll set sequences to be at most 128 tokens long.
merged_label_list = None
MAX_SEQ_LENGTH = 256
# Convert our train and test features to InputFeatures that BERT understands.

train_features = convert_examples_to_features(
train_InputExamples, merged_label_list, MAX_SEQ_LENGTH, tokenizer)

test_features = convert_examples_to_features(
test_InputExamples, merged_label_list, MAX_SEQ_LENGTH, tokenizer)


def create_model(is_predicting, input_ids, input_mask, segment_ids, labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  
  hidden_size = output_layer.shape[-1].value
  
  output_weights = tf.get_variable(
      "output_weights", [1, hidden_size],
  initializer=tf.truncated_normal_initializer(stddev=0.02))
  
  output_bias = tf.get_variable(
  "output_bias", [1], initializer=tf.zeros_initializer())
  
 
  with tf.variable_scope("loss"):
    if not is_predicting:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    logits = tf.squeeze(logits, [-1])

    per_example_loss = tf.square(logits - labels)
    
    # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits) = create_model(not is_training,
        input_ids, input_mask, segment_ids, label_ids)

    tvars = tf.trainable_variables()

    scaffold_fn = None
    if init_checkpoint:
      (assignment_map,
       initialized_variable_names) = modeling.get_assigment_map_from_checkpoint(
           tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

#     tf.logging.info("**** Trainable Variables ****")
#     for var in tvars:
#       init_string = ""
#       if var.name in initialized_variable_names:
#         init_string = ", *INIT_FROM_CKPT*"
#       tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
#                       init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

#       def metric_fn(per_example_loss, label_ids, logits):
#         predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
#         accuracy = tf.metrics.accuracy(label_ids, predictions)
#         loss = tf.metrics.mean(per_example_loss)
#         return {
#             "eval_accuracy": accuracy,
#             "eval_loss": loss,
#         }
      def metric_fn(per_example_loss, label_ids, logits):       
        # Display labels and predictions
        concat1 = tf.contrib.metrics.streaming_concat(logits)
        concat2 = tf.contrib.metrics.streaming_concat(label_ids)
        
        # Compute Pearson correlation
        
#         pearson = tf.contrib.metrics.streaming_pearson_correlation(logits, label_ids)
        
        # Compute MSE
        # mse = tf.metrics.mean(per_example_loss)    
        mse = tf.metrics.mean_squared_error(label_ids, logits)
        
        # Compute Spearman correlation
#         size = tf.size(logits)
#         indice_of_ranks_pred = tf.nn.top_k(logits, k=size)[1]
#         indice_of_ranks_label = tf.nn.top_k(label_ids, k=size)[1]
#         rank_pred = tf.nn.top_k(-indice_of_ranks_pred, k=size)[1]
#         rank_label = tf.nn.top_k(-indice_of_ranks_label, k=size)[1]
#         rank_pred = tf.to_float(rank_pred)
#         rank_label = tf.to_float(rank_label)
        
#         check here
#         spearman = tf.contrib.metrics.streaming_pearson_correlation(rank_pred, rank_label)
        
#         return {'pred': concat1, 'label_ids': concat2, 'pearson': pearson, 'spearman': spearman, 'MSE': mse}
        return {'pred': concat1, 'label_ids': concat2, 'MSE': mse}

      eval_metrics = metric_fn(per_example_loss, label_ids, logits)
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metric_ops=eval_metrics)
    else:
      raise ValueError("Only TRAIN and EVAL modes are supported: %s" % (mode))

    return output_spec

  return model_fn

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 6
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    batch_size = BATCH_SIZE

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            # tf.constant(all_label_ids, shape=[num_examples, 0], dtype=tf.float32),
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.float32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn


# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify outpit directory and number of checkpoint steps to save


#  run_config = tf.contrib.tpu.RunConfig(
#       model_dir=OUTPUT_DIR,
#       save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
#       )


run_config = tf.estimator.RunConfig(

    model_dir=OUTPUT_DIR,

    save_summary_steps=SAVE_SUMMARY_STEPS,

    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

model_fn = model_fn_builder(
    None, False, LEARNING_RATE,
    num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps, use_tpu=False,
    use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


Next we create an input builder function that takes our training feature set (train_features) and produces a generator. This is a pretty standard design pattern for working with Tensorflow Estimators.

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

**Now** we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

test_input_fn = input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

estimator.evaluate(input_fn=test_input_fn, steps=None)